# Dinh et al. (2019)

## A data-driven approach to predicting diabetes and cardiovascular disease with machine learning

URL: https://bmcmedinformdecismak.biomedcentral.com/articles/10.1186/s12911-019-0918-5


## Brief Summary

Dinh et al. (2019) uses different ML models (logistic regression, support vector machines, random forest, and gradient boosting) on NHANES dataset to predict i) Diabetes and ii) Cardiovascular disease ("CVD").

**Goal**: Identification mechanism for patients at risk of diabetes and cardiovascular diseases and key contributors to diabetes .

**Results**:

Best scores:

- CVB prediction based on 131 NHANES variables achieved an AU-ROC score of 83.9% .
- Diabetes prediction based on 123 NHANES variables achieved an AU-ROC score of 95.7% .
- Pre-diabetic prediction based on 123 NHANES variables achieved an AU-ROC score of 84.4% .
- Top 5 predictors in diabetes patients were 1) `waist size`, 2) `age`, 3) `self-reported weight`, 4) `leg length`, 5) `sodium intake`.



This notebook replicates the results of the paper. The structure follows the following steps: 

1. NHANES data 
2. Pre-processing of the data
3. Transformation of the data
4. Train/Test Split 
5. CV 10-fold
6. Training monitoring using MLflow
7. Get metric results (AUC)


The structure of the analysis emulates the Figure 1 from the paper: 

![Fig 1 from Dinh et al. 2019](https://raw.githubusercontent.com/pipegalera/ml_diabetes/main/images/dinh_2019_Fig1.png)


In [ ]:
library(arrow)
library(dplyr)
library(readxl)

## 1. HNANES data

URL: https://www.cdc.gov/nchs/index.htm


## Target

From the paper, the definitions are clear: 

![Dinh et al.(2019), Table 4](https://raw.githubusercontent.com/pipegalera/ml_diabetes/main/images/dinh_2019_Table4.png)

- Case I: Diabetes.

    - Glucose >= 126 mg/dL. OR;
    - "Yes" to the question "Have you ever been told by a doctor that you have diabetes?"

- Case II: Undiagnosed Diabetes. 

    - Glucose >= 126 mg/dL. AND;
    - "No" to the question "Have you ever been told by a doctor that you have diabetes?"

- Cardio: Cardiovascular disease.

    - "Yes" to any of the the questions "Have you ever been told by a doctor that you had congestive heart failure, coronary heart disease, a heart attack, or a stroke?"

The paper also defined and test for the target: 

- Pre diabetes

    - Glucose 125 >= 100 mg/dL

## Covariates

The paper did not say what variables they use from NHANES, so I emailed the author in the correspondence section of the paper to help me clarify that. No answer from him.

Given that NHANES have more than 3000 variables, I cannot just randomly take the variables I believe are important. 

For now, I will consider the variables taken from [Figure 5](https://raw.githubusercontent.com/pipegalera/ml_diabetes/main/images/dinh_2019_Fig5.png) and [Figure 6](https://raw.githubusercontent.com/pipegalera/ml_diabetes/main/images/dinh_2019_Fig6.png) of the paper. I compiled them by hand in an Excel file using NHANES search tool for variables:



In [11]:
DATA_PATH <- "/Users/pipegalera/dev/ml_diabetes/data/NHANES/raw_data/"
dinh_2019_vars <- read_excel(paste0(DATA_PATH, "dinh_2019_variables_doc.xlsx"))

print(dinh_2019_vars[, c("Variable Name", "NHANES Name")], n=62)


# A tibble: 62 x 2
   `Variable Name`                           `NHANES Name`
   <chr>                                     <chr>        
 1 "Age"                                     RIDAGEYR     
 2 "Alcohol consumption"                     ALQ130       
 3 "Alcohol intake"                          DRXTALCO     
 4 "Alcohol intake, First Day"               DR1TALCO     
 5 "Alcohol intake, Second Day"              DR2TALCO     
 6 "Arm circumference"                       BMXARMC      
 7 "Arm length"                              BMXARML      
 8 "Blood osmolality"                        LBXSOSSI     
 9 "Blood relatives have diabetes"           MCQ250A      
10 "Blood urea nitrogen"                     LBDSBUSI     
11 "BMI"                                     BMXBMI       
12 "Caffeine intake"                         DRXTCAFF     
13 "Caffeine intake, First Day"              DR1TCAFF     
14 "Caffeine intake, Second Day"             DR2TCAFF     
15 "Calcium intake, First Day"       

The data was downloaded automatically, all the files converted from SAS to parquet, the files stacked and merged based on the individual index ("SEQN"). For more details please check the `create_data_master.ipynb` notebook. Plese notice that no transformation are made to the covariates, only stacked them together. 

In [17]:
df <- read_parquet(paste0(DATA_PATH, "dinh_raw_data.parquet"))
head(df)

SEQN,YEAR,RIDAGEYR,ALQ130,DRXTALCO,DR1TALCO,DR2TALCO,BMXARMC,BMXARML,LBXSOSSI,...,BPXSY1,BPXSY4,BPXSY2,BPXSY3,LBDTCSI,LBDSTRSI,BMXWAIST,BMXWT,LBXWBCSI,LBXSASSI
<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1999-2000,2,NA,5.397605e-79,NA,NA,15.2,18.6,NA,...,NA,NA,NA,NA,NA,NA,45.7,12.5,NA,NA
2,1999-2000,77,1,5.397605e-79,NA,NA,29.8,38.2,288,...,106,NA,98,98,5.56,1.298,98.0,75.4,7.6,19
3,1999-2000,10,NA,5.397605e-79,NA,NA,19.7,25.5,NA,...,110,NA,104,112,3.34,NA,64.7,32.9,7.5,NA
4,1999-2000,1,NA,5.397605e-79,NA,NA,16.4,20.4,NA,...,NA,NA,NA,NA,NA,NA,NA,13.3,8.8,NA
5,1999-2000,49,3,3.456000e+01,NA,NA,35.8,39.7,276,...,122,NA,122,122,7.21,3.850,99.9,92.5,5.9,22
6,1999-2000,19,NA,5.397605e-79,NA,NA,26.0,34.5,277,...,116,NA,116,112,3.96,0.553,81.6,59.2,9.6,20


In [18]:
tail(df)

SEQN,YEAR,RIDAGEYR,ALQ130,DRXTALCO,DR1TALCO,DR2TALCO,BMXARMC,BMXARML,LBXSOSSI,...,BPXSY1,BPXSY4,BPXSY2,BPXSY3,LBDTCSI,LBDSTRSI,BMXWAIST,BMXWT,LBXWBCSI,LBXSASSI
<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
102951,2017-2018,4,NA,NA,5.397605e-79,5.397605e-79,20.9,24.0,NA,...,NA,NA,NA,NA,NA,NA,62.2,23.8,8.5,NA
102952,2017-2018,70,NA,NA,5.397605e-79,5.397605e-79,25.1,32.6,288,...,136,NA,142,140,3.08,1.106,82.2,49.0,5.1,27
102953,2017-2018,42,12,NA,5.397605e-79,NA,40.6,36.6,289,...,124,NA,122,116,4.71,1.287,114.8,97.4,8.3,29
102954,2017-2018,41,NA,NA,5.397605e-79,5.397605e-79,26.8,35.2,272,...,116,NA,118,114,4.45,0.723,86.4,69.1,5.1,15
102955,2017-2018,14,NA,NA,5.397605e-79,5.397605e-79,44.5,35.0,274,...,114,NA,114,114,3.88,1.005,113.5,111.9,11.4,16
102956,2017-2018,38,2,NA,5.397605e-79,5.397605e-79,40.0,38.0,277,...,150,NA,146,148,4.22,3.263,122.0,111.5,9.0,27


# 2. Preprocess the data 

The first step is filter the data:


> In our study, all datasets were limited to non-pregnant subjects and adults of at least twenty years of age.

> This lead to a further reduction in number of available variables to 123 for the 1999-2014 cycle.


In [43]:
df$SEQN <- as.integer(df$SEQN)
df$RIDAGEYR <- as.integer(df$RIDAGEYR)
df$RIAGENDR <- as.integer(df$RIAGENDR)


In [41]:
filtered_df <- df |> 
  filter(RHD143 == 2) |>  # Are you pregnant now? = "No"
  filter(RIDAGEYR >= 20) |>
  filter(YEAR != "2015-2016/")

> The data was further analyzed for missing values within the variables, and any with more than 50% of missing values were dropped from the dataset.

Since the pregnancy question (RHD143) was only for *Females only 20 YEARS - 44 YEARS*, the missing values are men or women out of that age range, so I will not delete them.

In [51]:
missing_values <- sapply(df, function(x) mean(is.na(x))) * 100
missing_values_df <- data.frame(missing_percentage = missing_values) |>
                        arrange(desc(missing_percentage))
print(missing_values_df)

         missing_percentage
RHD143            93.640156
BPXDI4            91.407964
BPXSY4            91.407964
LBDLDL            80.323043
LBDTRSI           80.009124
LBDGLUSI          79.577893
LBXGH             57.676349
BPXDI3            33.969499
BPXSY3            33.968413
BPXDI2            33.337316
BPXSY2            33.337316
BPXDI1            32.356455
BPXSY1            32.356455
BPXPLS            27.077404
BMXWAIST          16.254263
BMXBMI            13.325802
BMXHT             12.859812
BMXWT              5.930786
DIQ010             4.515435
SEQN               0.000000
RIDAGEYR           0.000000
RIAGENDR           0.000000
YEAR               0.000000


In [ ]:



seeing if there is any extreme value that might be due to bad input of the data. According to the paper:

> The preprocessing stage also converted any undecipherable values (errors in datatypes and standard formatting) from the database to null representations.

In [ ]:
boxplot(df[, c('col1', 'col2', 'colN')])


In [ ]:
normalize_columns <- function(master, columns) {
  for (column in columns) {
    master[, column] <- (master[, column] - min(master[, column])) / (max(master[, column]) - min(master[, column]))
  }
  return(master)
}
